# Comparison Operators — Practice Notebook

Hands-on exercises for:
- Equality vs identity: `==` vs `is`
- Ordering (`<`, `<=`, `>`, `>=`) and chained comparisons
- Container comparisons (lists/tuples/sets/dicts)
- Floats and `math.isclose`, `nan` gotchas
- Custom types: implementing equality & ordering
- Membership: `in` / `not in`


## 1) Equality (`==`) vs Identity (`is`)
- Use `==` to compare **values**.
- Use `is` only for **object identity** (e.g., checking against `None`).
- Do **not** rely on `is` for numbers/strings; interning is an implementation detail.


In [1]:
a = [1, 2, 3]
b = [1, 2, 3]
c = a

assert (a == b) is True      # same contents
assert (a is b) is False     # different list objects
assert (a is c) is True      # same object

x = None
assert (x is None) is True   # preferred way to check None
print("OK ✅")


OK ✅


## 2) Container equality & ordering
- Lists/tuples compare elementwise; tuples/lists are ordered.
- Sets compare by members (order-insensitive); dicts compare by key/value pairs.
- Tuples compare **lexicographically**.


In [2]:
# Lists / tuples: elementwise equality
assert [1, 2] == [1, 2] and (1, 2) == (1, 2)
assert [1, 2] != [2, 1]  # order matters for lists/tuples

# Sets: order-insensitive / unique elements by equality
assert {1, 2, 3} == {3, 2, 1}

# Dicts: compare by key-value pairs
assert {"a": 1, "b": 2} == {"b": 2, "a": 1}

# Tuples: lexicographic ordering
assert (1, 10) < (2, 0)    # 1 < 2 so True
assert (1, 10) > (1, 2)    # first equal, 10 > 2
print("OK ✅")


OK ✅


## 3) Chained comparisons
`a < b < c` is equivalent to `(a < b) and (b < c)` with `b` evaluated **once**. Mixed chains like `a < b == c` are allowed.


In [3]:
calls = 0
def mid():
    global calls
    calls += 1
    return 10

calls = 0
assert (5 < mid() < 20) is True and calls == 1

# Mixed comparison chain
a, b, c = 3, 3, 3
assert (2 < b == c <= 3) is True
print("OK ✅")


OK ✅


## 4) Floats: avoid `==` — use `math.isclose`
- Most decimal fractions are not exactly representable in binary floating point.
- Use `math.isclose` with appropriate tolerances.
- `nan` is never equal to anything, including itself.


In [4]:
import math

assert (0.1 * 3 == 0.3) is False
assert math.isclose(0.1 * 3, 0.3, rel_tol=1e-12, abs_tol=0.0)

nan = float('nan')
assert (nan == nan) is False
assert math.isnan(nan) is True
print("OK ✅")


OK ✅


## 5) Custom types: equality & ordering
Use `functools.total_ordering` to derive full ordering from `__eq__` and one other method (e.g., `__lt__`).


In [5]:
from functools import total_ordering

@total_ordering
    
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        return f"Vector({self.x}, {self.y})"
    def __eq__(self, other):
        if isinstance(other, Vector):
            return (self.x, self.y) == (other.x, other.y)
        return NotImplemented
    def __lt__(self, other):
        if isinstance(other, Vector):
            # order by length squared
            return (self.x**2 + self.y**2) < (other.x**2 + other.y**2)
        return NotImplemented

v1, v2, v3 = Vector(1,1), Vector(1,1), Vector(2,3)
assert (v1 == v2) is True
assert (v1 is v2) is False
assert (v1 < v3) is True
assert sorted([v3, v1, v2])[0] == v1
print("OK ✅", sorted([v3, v1, v2]))


OK ✅ [Vector(1, 1), Vector(1, 1), Vector(2, 3)]


## 6) Membership: `in` / `not in`
- Sequences use equality to test membership.
- Sets/dicts use hash + equality; order doesn’t matter for sets.
- Remember: `True == 1` and `False == 0`, which can affect sets.


In [6]:
seq = ["cat", "dog", "fox"]
assert ("dog" in seq) is True
assert ("cow" not in seq) is True

# Set quirk: True and 1 are equal, so they collide in a set
    
s = {0, 1, True, False}
assert len(s) == 2   # {0, 1}
assert (True in s) and (1 in s)

# Dict membership checks keys by default
d = {"a": 1, "b": 2}
assert ("a" in d) and (1 not in d)
print("OK ✅")


OK ✅


## 7) Edge cases & best practices
- Prefer `is` only for `None`, `True`, `False` **identity checks** (not value comparisons).
- Do not depend on identity of small integers/strings (implementation detail).
- For floats, prefer `math.isclose` or domain-specific tolerances.
- For custom classes, return `NotImplemented` when comparing to unsupported types.


## 8) Quick quiz — predict before running


In [7]:
# A) Equality vs identity
t1 = (1, 2, 3)
t2 = (1, 2, 3)
assert (t1 == t2) is True and (t1 is t2) is False

# B) Chained comparisons
x, y, z = 2, 3, 3
assert (1 < x < y <= z) is True

# C) Lexicographic tuple order
assert ("a", 10) < ("b", 0)
assert ("a", 10) > ("a", 2)

# D) Sets and equality
assert {1, 2, 3} == {3, 2, 1} and {1, 2} != {1, 2, 3}

# E) Floats and NaN
import math
v = float('nan')
assert (v == v) is False and math.isnan(v) is True
assert (0.1 + 0.2 == 0.3) is False
assert math.isclose(0.1 + 0.2, 0.3, rel_tol=1e-12)

print("All quiz checks passed ✅")


All quiz checks passed ✅
